<a href="https://colab.research.google.com/github/tafakuri/amazon-sagemaker-fine-tune-and-deploy-wav2vec2-huggingface/blob/main/colab/Evaluate_WER_for_model_with_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture cap --no-stderr
!pip install datasets
!pip install transformers
!pip install torch
!pip install torchaudio

#!pip install jiwer==2.2.0
#!pip install lang-trans==0.6.0
#!pip install librosa==0.9.1
!pip install jiwer
!pip install lang-trans
!pip install librosa
!pip install huggingface_hub
!pip install pyctcdecode

In [2]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
%%capture
!apt install git-lfs

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     - 553.3 kB 8.1 MB/s 0:00:00


This block was added when "import kenlm" call failed. The call failed even after this was executed, and the true fix was to restart the colab runtime. 

TODO: Remove (or run without) this block and see if the full evaluation process still works.

In [ ]:
! sudo apt -y install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev
! wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz
! mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
! ls kenlm/build/bin

Evaluate model WER

In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import Wav2Vec2ProcessorWithLM
import re
import kenlm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

test_dataset = load_dataset("mozilla-foundation/common_voice_9_0", "sw", split="test",use_auth_token=True)
wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("mutisya/fine-tune-xls-r-300m-wav2vec2-on-swahili-sagemaker-withLM")
processor.feature_extractor.return_attention_mask =True
model = Wav2Vec2ForCTC.from_pretrained("mutisya/fine-tune-xls-r-300m-wav2vec2-on-swahili-sagemaker-withLM")
model.to(device)

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'  # TODO: adapt this list to include all special characters you removed from the data
#chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"áéíóúñ°öøµëïâãšóụūåã•]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
	batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
	speech_array, sampling_rate = torchaudio.load(batch["path"])
	batch["speech"] = resampler(speech_array).squeeze().numpy()
	return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)


In [ ]:

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
  inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
  """
  with torch.no_grad():
    logits = model(**inputs).logits
  """

  with torch.no_grad():
  	logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits

  transcription = processor.batch_decode(logits.numpy()).text

  batch["pred_strings"] = transcription
  return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

In [ ]:
def evaluate_2(batch):
  inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
  """
  with torch.no_grad():
    logits = model(**inputs).logits
  """
  
  with torch.no_grad():
  	logits = model(inputs.input_values.to(device), attention_mask=inputs.attention_mask.to(device)).logits

  #transcription = processor.batch_decode(logits.numpy()).text

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_strings"] = processor.batch_decode(pred_ids)
  # batch["pred_strings"] = transcription
  return batch

#result = test_dataset.map(evaluate, batched=True, batch_size=8)

#print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

# Here is the function used during training
wer_metric = load_metric("wer") 
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

